In [ ]:
import numpy as np
import os
import base64
import re
import sqlite3
import time
from datetime import datetime
from flask import Flask, request, jsonify, g
from tensorflow.keras.models import load_model # load_model para cargar todo de una vez
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import io
import sys
import requests
from PIL import Image
import json
import cv2



In [ ]:
# Prueba que dio tantos problemas, la predicción de la imagen
# endpoint def predict (predice con el modelo de luna, pero no devuelve nada en local (no sé si se puede hacer un post + get a la vez en el mismo endpoint,
#  me suena que en clase vimos uno pero haciendo las request por separado))

image_path = r"..\Mini_base_datos\TCGA_CS_4941_19960909_14.tif"

with open(image_path, 'rb') as f:
    response = requests.post(
        'http://127.0.0.1:5000/predict',
        files={'image': (image_path, f, 'image/tiff')}
    )

print("Status:", response.status_code)
print("Respuesta:", response.text)

In [ ]:
# La predicción de la imagen enviada en la celda anterior

# 1. POST - enviar imagen

with open(image_path, 'rb') as f:
    post_response = requests.post(
        'http://127.0.0.1:5000/predict',
        files={'image': (image_path, f, 'image/tiff')}
    )

post_result = post_response.json()
print("POST:", post_result)

# 2. GET - consultar la misma predicción automáticamente
prediction_id = post_result['prediction_id']
get_response = requests.get(f'http://127.0.0.1:5000/predict/{prediction_id}') #aquí pondiramos el id de la imagen de la celda anterior
print("GET:", get_response.json())

In [ ]:
# Endpoint predict random (devuelve el JSON sin imagen)
response = requests.get('http://127.0.0.1:5000/predict/random')
print(response.json())

In [ ]:
# Endpoint predict random view, devuelve la predicción (JSON + imagen), para visualizar la imagen nos hemos metido en el campo de front, si queremos cambiarlo sería necesario decodificar la imagen

response = requests.get('http://127.0.0.1:5000/predict/random/view')
print(response.text)

In [ ]:
base_url = 'http://127.0.0.1:5000'
# es la def get_history, es para ver el historial de las predicciones previas

# GET - Ver historial
response = requests.get(base_url + '/history')
print("Status:", response.status_code)
print("Response:", response.text)

In [ ]:
# def upgate_feedback --> corrige una predicción, por si la que veis en la celda posterior fuese necesaria. Son ambas más de adorno que otra cosa, pero no se nos ocurren más.
# PUT - Corregir predicción
feedback = {"correct_label": "Clase_1"}

response = requests.put(base_url + '/feedback/1', json=feedback)
print("Status:", response.status_code)
print("Response:", response.text)

In [ ]:
# def train_model ---> serviría para reentrenar el modelo

base_url = 'http://127.0.0.1:5000'

train_config = {"epochs": 10, "learning_rate": 0.0005}

response = requests.post(base_url + '/admin/train', json=train_config)
print("Status:", response.status_code)
print("Response:", response.text)